[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Agents/KlavisAI/Use_Klavis_with_Together.ipynb)


# Together AI + Klavis AI Integration

<img src="../../images/klavis.png" width="700">

In this tutorial, we'll explore how to build an AI agent that integrates Together AI's powerful LLMs with Klavis MCP Servers, enabling seamless interaction with external services and APIs.

This integration combines:
- **Together AI**: High-performance open-source LLMs with function calling capabilities
- **Klavis AI**: MCP (Model Context Protocol) servers for connecting to external tools and services


## Prerequisites

Before we begin, you'll need:

- **Together AI API key** - Get yours at [together.ai](https://together.ai/)
- **Klavis AI API key** - Get yours at [klavis.ai](https://klavis.ai/)

Make sure to keep these API keys secure and never commit them to version control!


In [1]:
# Install the required packages
%pip install -qU together klavis


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import json
from together import Together
from klavis import Klavis
from klavis.types import McpServerName, ConnectionType, ToolFormat

# Set environment variables
os.environ["TOGETHER_API_KEY"] = "your-together-api-key-here"  # Replace with your actual Together API key
os.environ["KLAVIS_API_KEY"] = "your-klavis-api-key-here"      # Replace with your actual Klavis API key

## Create AI Agent with MCP Integration

Now we'll create an intelligent agent that uses Together AI's powerful LLMs with Klavis MCP servers. This agent will:

1. **Discover Tools**: Automatically find available tools from MCP servers
2. **Function Calling**: Use Together AI's function calling capabilities
3. **Tool Execution**: Execute tools through Klavis API
4. **Smart Responses**: Generate intelligent responses based on tool results


In [7]:
class Agent:
    def __init__(self, together_client, klavis_client, mcp_server_url, model="meta-llama/Llama-3.3-70B-Instruct-Turbo"):
        self.together = together_client
        self.klavis = klavis_client
        self.mcp_server_url = mcp_server_url
        self.model = model
        print(f"🤖 Agent initialized with Together AI model: {self.model}")
    
    def process_request(self, user_message):
        # 1. Get available tools
        mcp_tools = self.klavis.mcp_server.list_tools(
            server_url=self.mcp_server_url,
            connection_type=ConnectionType.STREAMABLE_HTTP,
            format=ToolFormat.OPENAI,
        )
        
        # 2. Call LLM with tools
        messages = [
            {"role": "system", "content": "You are a helpful AI assistant with access to various tools."},
            {"role": "user", "content": user_message}
        ]
        
        response = self.together.chat.completions.create(
            model=self.model,
            messages=messages,
            tools=mcp_tools.tools
        )
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        
        # 3. If LLM wants to use tools
        if assistant_message.tool_calls:
            
            # Execute each tool call
            for tool_call in assistant_message.tool_calls:
                tool_name = tool_call.function.name
                tool_args = json.loads(tool_call.function.arguments)
                
                print(f"🛠️ Calling tool: {tool_name} with args: {tool_args}")
                # Call tool via Klavis SDK
                tool_result = self.klavis.mcp_server.call_tools(
                    server_url=self.mcp_server_url,
                    tool_name=tool_name,
                    tool_args=tool_args,
                    connection_type=ConnectionType.STREAMABLE_HTTP
                )
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(tool_result)
                })
            
            # 4. Get final response from LLM
            final_response = self.together.chat.completions.create(
                model=self.model,
                messages=messages
            )
            return final_response.choices[0].message.content
        
        # If no tools needed, return the assistant message directly
        return assistant_message.content


## Use Case 1: Summarize YouTube Video

In [8]:

# Example YouTube video URL - replace with any video you'd like to analyze
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=TG6QOa2JJJQ"

# 1. Initialize Together AI client and Klavis client
together_client = Together(api_key=os.getenv("TOGETHER_API_KEY"))
klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

# 2. Create YouTube MCP server instance
youtube_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.YOUTUBE,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

# 3. Create an agent with YouTube MCP server
agent = Agent(
    together_client=together_client, 
    klavis_client=klavis_client, 
    mcp_server_url=youtube_mcp_instance.server_url,
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo"
)

# 4. Process the request
response = agent.process_request(
    f"Please analyze this YouTube video and provide a comprehensive summary with timestamps: {YOUTUBE_VIDEO_URL}"
)

print(response)

✅ Created YouTube MCP instance
🤖 Agent initialized with Together AI model: meta-llama/Llama-3.3-70B-Instruct-Turbo
🛠️ Calling tool: get_youtube_video_transcript with args: {'url': 'https://www.youtube.com/watch?v=TG6QOa2JJJQ'}
The YouTube video "Together AI CEO: Open Source Is the Future of AI" features an interview with Vipul Ved Prakash, the CEO of Together AI, and Bucky Moore, a partner at Kleiner Perkins. The discussion revolves around Together AI's $102.5 million Series A fundraise, led by Kleiner Perkins, and the company's focus on open-source AI.

Here is a comprehensive summary of the video with timestamps:

* 0:00 - Introduction to the video and the guests
* 0:30 - Discussion of Together AI's Series A fundraise and the company's mission
* 1:45 - Vipul Ved Prakash explains the importance of open-source AI and how it can benefit the industry
* 3:10 - Bucky Moore shares his perspective on the potential of open-source AI and its applications
* 4:30 - The guests discuss the challen

## Use Case 2: Send Email

**Note**: Gmail integration requires OAuth authentication, so you'll need to authorize the application in your browser.




In [9]:
import webbrowser

# Create Gmail MCP server instance
gmail_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.GMAIL,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

# Redirect to Gmail OAuth page for authorization
webbrowser.open(gmail_mcp_instance.oauth_url)
print(f"🔐 Opening OAuth authorization for Gmail")
print(f"If you are not redirected automatically, please open this URL: {gmail_mcp_instance.oauth_url}")


✅ Created Gmail MCP instance
🔐 Opening OAuth authorization for Gmail
If you are not redirected automatically, please open this URL: https://api.klavis.ai/oauth/gmail/authorize?instance_id=d9d482b3-433a-4330-9a8b-9548c0b0a326


In [10]:
# Email configuration
EMAIL_RECIPIENT = "zihaolin@klavis.ai"  # Replace with the recipient's email
EMAIL_SUBJECT = "Greetings from Together AI + Klavis Integration"
EMAIL_BODY = "This is a test email sent using the Together AI and Klavis AI integration. The email was sent automatically by your AI agent!"

# After OAuth authorization is complete, create the Gmail agent
gmail_agent = Agent(
    together_client=together_client,
    klavis_client=klavis_client,
    mcp_server_url=gmail_mcp_instance.server_url,
    model="Qwen/Qwen2.5-72B-Instruct-Turbo"
)

# Send the email
response = gmail_agent.process_request(
    f"Please send an email to {EMAIL_RECIPIENT} with the subject '{EMAIL_SUBJECT}' and the following body: '{EMAIL_BODY}'"
)

print(response)


🤖 Agent initialized with Together AI model: Qwen/Qwen2.5-72B-Instruct-Turbo
🛠️ Calling tool: send_email with args: {'body': 'This is a test email sent using the Together AI and Klavis AI integration. The email was sent automatically by your AI agent!', 'subject': 'Greetings from Together AI + Klavis Integration', 'to': ['zihaolin@klavis.ai']}
The email has been sent successfully to zihaolin@klavis.ai with the subject 'Greetings from Together AI + Klavis Integration'. The email ID is 19776f818ce706db.


## 🎉 Conclusion

Congratulations! You've successfully built AI agents that integrate Together AI's powerful LLMs with Klavis MCP servers. Here's what we accomplished:

### 🚀 Next Steps
- **Explore More MCP Servers**: Try other available servers like Slack, Notion, CRM etc.
- **Experiment with Different Models**: Test various Together AI models for different use cases.
- **Build Complex Multi-Server Workflows**: Create sophisticated agents that combine multiple services
- **Production Deployment**: Scale these patterns for production applications
- **Custom MCP Servers**: Build your own MCP servers for proprietary systems

### 🔗 Useful Resources
- [Together AI Documentation](https://docs.together.ai/)
- [Klavis AI Documentation](https://docs.klavis.ai/)
- [MCP Protocol Specification](https://modelcontextprotocol.io/)
- [Together AI Models](https://docs.together.ai/docs/inference-models)
- [Klavis MCP Servers](https://docs.klavis.ai/mcp-servers)

**Happy building with Together AI and Klavis!** 🚀
